In [ ]:
import torch
import torch.optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt


def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def get_data(path, leng_data):
    dataset = unpickle(path)
    data_x = [data for index, data in enumerate(dataset[b'data'])]
    data_x = np.array(data_x)
    data_y = [data for index, data in enumerate(dataset[b'labels'])]
    data_y = np.array(data_y)
    # print(data_x.shape)
    # print(data_y.shape)
    
    data_x_red=data_x[0:leng_data,0:1024]
    data_x_green=data_x[0:leng_data,1024:2048]
    data_x_blue=data_x[0:leng_data,2048:]

    red=data_x_red.reshape(len(data_x_red),32,32)
    green=data_x_green.reshape(len(data_x_green),32,32)
    blue=data_x_blue.reshape(len(data_x_blue),32,32)

    sample=np.stack([red, green, blue], axis=1)
    sample=sample.astype(np.float32) / 255.0
    sample=sample.reshape(len(sample),3,32,32)
    # print(len(sample))
    label=data_y[0:leng_data]
    # print(sample.shape)
    # print(label.shape)
    
    return sample, label

class block(torch.nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(block, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), stride=stride, padding=(1, 1), bias=False)
        self.Bn1 = torch.nn.BatchNorm2d(out_channels)
        self.Ru = torch.nn.ReLU(inplace=True)
        self.conv2 = torch.nn.Conv2d(out_channels, out_channels, kernel_size=(3, 3), stride=1, padding=(1, 1), bias=False)
        self.Bn2 = torch.nn.BatchNorm2d(out_channels)

        self.connection = torch.nn.Identity()
        if in_channels != out_channels:
            self.connection = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels, out_channels, kernel_size=(1, 1), stride=stride, bias=False),
                torch.nn.BatchNorm2d(out_channels)
            )
        else:
            self.connection = torch.nn.Sequential(
                torch.nn.Conv2d(out_channels, out_channels, kernel_size=(1, 1), stride=1, bias=False),
                torch.nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.connection(x)
        x = self.Ru(self.Bn1(self.conv1(x)))
        x = self.Bn2(self.conv2(x)) + identity
        return self.Ru(x)
    
class layers(torch.nn.Module):
    def __init__(self, layer, in_channels, out_channels, stride):
        super(layers,self).__init__()
        self.layer=layer
        self.in_channels=in_channels
        self.out_channels=out_channels
        self.stride=stride
        self.Layer = self._Layer()
    def _Layer(self):
        layers=[]
        for _ in range(self.layer):
            layers.append(block(self.in_channels, self.out_channels, self.stride))
            self.in_channels=self.out_channels
            self.stride=1
        return torch.nn.Sequential(*layers)
    def forward(self, x):
        return self.Layer(x)        

class Resnet(torch.nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        self.in_channel = 3
        self.kernel_size=(3,3)
        self.stride = [1,2]
        self.padding = (1, 1)
        self.kernel_size1=(7,7)
        self.padding2 = (3, 3)
        self.Ci = [64, 128, 256, 512]

        self.conv1 = torch.nn.Conv2d(in_channels=self.in_channel, out_channels=self.Ci[0], kernel_size=self.kernel_size, stride=self.stride[0], padding=self.padding, bias=True)
        # self.conv1 = torch.nn.Conv2d(in_channels=self.in_channel, out_channels=self.Ci[0], kernel_size=self.kernel_size1, stride=self.stride[1], padding=self.padding2, bias=False)
        self.ReLU = torch.nn.ReLU(inplace=True)
        self.BN = torch.nn.BatchNorm2d(self.Ci[0],eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.Averagepool2=torch.nn.MaxPool2d(kernel_size=self.kernel_size, stride=self.stride[1], padding=self.padding, dilation=1, ceil_mode=False)  
        
        self.layer1=layers(2,self.Ci[0], self.Ci[0], stride=self.stride[0])
        self.layer2=layers(2,self.Ci[0], self.Ci[1], stride=self.stride[1])
        self.layer3=layers(2,self.Ci[1], self.Ci[2], stride=self.stride[1])
        # self.layer4=layers(2,self.Ci[2], self.Ci[3], stride=self.stride[1])

        self.layer_pick=self.Ci[2]
        
        self.fc = torch.nn.Linear(self.layer_pick, 10, bias=True) 
        
        self.softmax = torch.nn.Softmax(dim=1)
        self.Averagepool=torch.nn.AdaptiveAvgPool2d(output_size=(1, 1))

        
    def forward(self, x):
        
        x = self.ReLU(self.BN(self.conv1(x)))
        # x = self.Averagepool2(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.layer4(x)

        x = self.Averagepool(x)

        x = x.view(-1,self.layer_pick)
        x = self.fc(x)
        x = self.softmax(x)

        return x
    
# from torchvision import datasets, models, transforms
# model = models.resnet34(pretrained=True).cuda()

model=Resnet().cuda()
loss=torch.nn.CrossEntropyLoss()
#opt = torch.optim.Adam(model.parameters(), lr=0.001, betas=[0.9,0.999],eps=1e-8)
opt=torch.optim.SGD(model.parameters(),lr=0.1,momentum=0.9 ,weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma=0.1)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(count_parameters(model))

train_loss_hist=[]
test_loss_hist=[]
accuracy_hist=[]

leng_data=10000

path=r"C:\Users\26435\Desktop\ECE-Gy Deep\Code_ex\Proj\deep-learning-spring-2025-project-1\cifar-10-python\cifar-10-batches-py\data_batch_1"
path3=r"C:\Users\26435\Desktop\ECE-Gy Deep\Code_ex\Proj\deep-learning-spring-2025-project-1\cifar-10-python\cifar-10-batches-py\data_batch_2"
path4=r"C:\Users\26435\Desktop\ECE-Gy Deep\Code_ex\Proj\deep-learning-spring-2025-project-1\cifar-10-python\cifar-10-batches-py\data_batch_3"
path5=r"C:\Users\26435\Desktop\ECE-Gy Deep\Code_ex\Proj\deep-learning-spring-2025-project-1\cifar-10-python\cifar-10-batches-py\data_batch_4"
path6=r"C:\Users\26435\Desktop\ECE-Gy Deep\Code_ex\Proj\deep-learning-spring-2025-project-1\cifar-10-python\cifar-10-batches-py\data_batch_5"
    
sample,label=get_data(path,leng_data)
batch1_x = torch.tensor(sample)
batch1_y = torch.tensor(label)
sample,label=get_data(path3,leng_data)
batch2_x = torch.tensor(sample)
batch2_y = torch.tensor(label)
sample,label=get_data(path4,leng_data)
batch3_x = torch.tensor(sample)
batch3_y = torch.tensor(label)
sample,label=get_data(path5,leng_data)
batch4_x = torch.tensor(sample)
batch4_y = torch.tensor(label)
sample,label=get_data(path6,leng_data)
batch5_x = torch.tensor(sample)
batch5_y = torch.tensor(label)
x_tensor = torch.cat([batch1_x, batch2_x,batch3_x,batch4_x,batch5_x], dim=0)
y_tensor = torch.cat([batch1_y, batch2_y,batch3_y,batch4_y,batch5_y], dim=0)

class transform(Dataset):
    def __init__(self, x_tensor, y_tensor, transform=None):
        self.x_tensor = x_tensor
        self.y_tensor = y_tensor
        self.transform = transform

    def __len__(self):
        return len(self.x_tensor)

    def __getitem__(self, index):
        x = self.x_tensor[index]
        y = self.y_tensor[index]

        if self.transform:
            x = self.transform(x)

        return x, y
    
train_transforms = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
test_transform = transforms.Compose([
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
dataset = transform(x_tensor, y_tensor, transform=train_transforms)

path2=r"C:\Users\26435\Desktop\ECE-Gy Deep\Code_ex\Proj\deep-learning-spring-2025-project-1\cifar-10-python\cifar-10-batches-py\test_batch"
sample,label=get_data(path2,leng_data)
x_tensor = torch.tensor(sample)
y_tensor = torch.tensor(label)
dataset = transform(x_tensor, y_tensor, transform=test_transform)

data_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)
data_loader_test = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False)

epoch_len=50
for edop in range(epoch_len):
    train_loss=0.0
    test_loss=0.0
    correct=0
    total_count=0
    model.train()
    for i,data in enumerate(data_loader):
        images,labels=data
        images=images.cuda()
        labels=labels.long().cuda()
        opt.zero_grad()
        predict_y=model(images)

        fit=loss(predict_y,labels)
        fit.backward()
        opt.step()
        train_loss+=fit.item()
        
    model.eval()
    for i,data in enumerate(data_loader_test):
        with torch.no_grad():
            images,labels=data
            images=images.cuda()
            labels=labels.long().cuda()
            predict_y=model(images)
            predicted_classes = torch.max(predict_y, 1)[1]
            fit=loss(predict_y,labels)
            test_loss+=fit.item()
            correct += (predicted_classes == labels).sum().item()
            total_count += labels.size(0)
            
    scheduler.step()
    
    final_accuracy=correct/total_count      
    
    train_loss=train_loss/len(data_loader)
    test_loss=test_loss/len(data_loader_test)
    accuracy_hist.append(final_accuracy)
    train_loss_hist.append(train_loss)
    test_loss_hist.append(test_loss)
    
    print(f'Epoch {edop},Train loss {train_loss},Test loss {test_loss},Final acc {final_accuracy}')

plt.figure()  
plt.plot(range(epoch_len),train_loss_hist,'-',linewidth=3,label='Train error')
plt.plot(range(epoch_len),test_loss_hist,'-',linewidth=3,label='Test error')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid(True)
plt.legend()
plt.show() 

plt.figure()  
plt.plot(range(epoch_len),accuracy_hist,'-',linewidth=3,label='accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.grid(True)
plt.legend()
plt.show() 

2868874
Epoch 0,Train loss 2.1219829979455076,Test loss 2.092039894454087,Final acc 0.3626
Epoch 1,Train loss 1.9886822066343655,Test loss 2.0197547447832327,Final acc 0.4322
Epoch 2,Train loss 1.8914640407123224,Test loss 1.8787319780905036,Final acc 0.5871
Epoch 3,Train loss 1.841606229162582,Test loss 1.8979330606098417,Final acc 0.5607
Epoch 4,Train loss 1.806178295094034,Test loss 1.8457725742195226,Final acc 0.6181
Epoch 5,Train loss 1.7749960574957415,Test loss 1.8510072020035755,Final acc 0.6125
Epoch 6,Train loss 1.7472906884025126,Test loss 1.7654166236708435,Final acc 0.6959
Epoch 7,Train loss 1.7290813547875874,Test loss 1.824456101731409,Final acc 0.6383
Epoch 8,Train loss 1.7192171751080876,Test loss 1.7575609880157663,Final acc 0.7064
Epoch 9,Train loss 1.7082259993419013,Test loss 1.73013076752047,Final acc 0.7397
Epoch 10,Train loss 1.6461067044216653,Test loss 1.6301151604592046,Final acc 0.8352
Epoch 11,Train loss 1.6268855693090298,Test loss 1.6196135086349295,Final

KeyboardInterrupt: 